In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import pickle
import datetime

In [2]:
df = pd.read_pickle('samsung_co_inventor_data(2013-2014).pkl')
co_inventor_df = df.groupby('patent_id')['inventor_id'].apply(lambda x: list(x)).reset_index()

In [3]:
abstract_df = pd.read_pickle('abstract_df.pkl')

In [4]:
relationship = []

for i in range(len(co_inventor_df)):
    co_inventor_list = co_inventor_df.ix[i]['inventor_id']
    for idx, data in  enumerate(co_inventor_list):
        for j in co_inventor_list[idx:]:
            if data != j:
                relationship.append((data, j))

In [5]:
inventor_node = df.drop_duplicates(subset='inventor_id', keep='first').inventor_id.tolist()

In [6]:
G = nx.MultiGraph()
G.add_nodes_from(inventor_node)
G.add_edges_from(relationship)

In [7]:
i = 0

while(1):
    print(i,"번째",datetime.datetime.utcnow())
    i += 1
    
    remove_edge_list = []
    edge_btw = nx.edge_betweenness_centrality(G)
    
    if len(set(edge_btw.values())) == 1:
        break
    
    else:
        for k,v in edge_btw.items():
            if v == max(edge_btw.values()):
                remove_edge_list.append(k)
        G.remove_edges_from(remove_edge_list)

0 번째 2017-03-10 05:10:36.581113
1 번째 2017-03-10 05:10:39.136773
2 번째 2017-03-10 05:10:41.689662
3 번째 2017-03-10 05:10:44.221616
4 번째 2017-03-10 05:10:46.789964
5 번째 2017-03-10 05:10:49.324360
6 번째 2017-03-10 05:10:51.806054
7 번째 2017-03-10 05:10:54.278771
8 번째 2017-03-10 05:10:56.753231
9 번째 2017-03-10 05:10:59.248065
10 번째 2017-03-10 05:11:01.754907
11 번째 2017-03-10 05:11:04.267579
12 번째 2017-03-10 05:11:06.766649
13 번째 2017-03-10 05:11:09.269845
14 번째 2017-03-10 05:11:11.753655
15 번째 2017-03-10 05:11:14.262834
16 번째 2017-03-10 05:11:16.762863
17 번째 2017-03-10 05:11:19.230507
18 번째 2017-03-10 05:11:21.764040
19 번째 2017-03-10 05:11:24.257672
20 번째 2017-03-10 05:11:26.800416
21 번째 2017-03-10 05:11:29.300798
22 번째 2017-03-10 05:11:31.792980
23 번째 2017-03-10 05:11:34.256687
24 번째 2017-03-10 05:11:36.653289
25 번째 2017-03-10 05:11:39.066093
26 번째 2017-03-10 05:11:41.480314
27 번째 2017-03-10 05:11:43.896452
28 번째 2017-03-10 05:11:46.311704
29 번째 2017-03-10 05:11:48.720185
30 번째 2017-03-10 05:

In [115]:
dict_connected_graph = nx.to_dict_of_lists(G)

del_key = []
for k,v in dict_connected_graph.items():
    if v == []:
        del_key.append(k)
        
for k in del_key:
    del dict_connected_graph[k]

inventor = set(inventor_node) - set(del_key)

In [116]:
def connected_components(nodes):
    
    result = []
    nodes = set(nodes)
            
    while nodes:
        
        n = nodes.pop()
        group = {n}
        queue = [n]

        while queue:

            n = queue.pop(0)
            neighbors = set(dict_connected_graph[n])

            neighbors.difference_update(group)
            nodes.difference_update(neighbors)
            group.update(neighbors)
            queue.extend(neighbors)
            
        result.append(group)
    
    return result

In [117]:
#inventor 그룹
all_group = []
number = 1
for components in connected_components(inventor):
    names = sorted(node for node in components)
    group = list(names)
    if len(group)>=3:
        all_group.append(group)
    number += 1

In [118]:
#inventor 그룹 마다 쓴 특허
all_patent_list = []
for i in range(len(all_group)):
    patent_set = set([])
    for j in range(len(all_group[i])):
        patent_set.update(df[df['inventor_id']==all_group[i][j]]['patent_id'].tolist())
    all_patent_list.append(list(patent_set))

In [119]:
#inventor 그룹 마다 쓴 특허의 abstract
all_patent_abstract = []
for i in range(len(all_patent_list)):
    abstract_set = set([])
    for j in range(len(all_patent_list[i])):
        abstract_set.update(abstract_df[abstract_df['patent_id']==all_patent_list[i][j]]['abstract'].tolist())
    all_patent_abstract.append(list(abstract_set))

In [120]:
#inventor가 쓴 특허가 3개 이상인 것 사용을 위해 삭제 해야 할 index 리스트 생성
drop_index_num = []
for i in range(len(all_patent_list)):
    if len(all_patent_list[i])<=2:
        drop_index_num.append(i)

In [121]:
#dict에 저장하기 위해 inventor, abstract 리스트 생성
inventor = []
for i in all_group:
    inventor.append("+".join(i))
    
abstract = []
for i in all_patent_abstract:
    abstract.append("\n".join(i))

In [137]:
#삭제 해야 할 inventor 리스트 생성
drop_inventor = []
for idx, data in enumerate(inventor):
    if (idx in drop_index_num)==True:
        drop_inventor.append(data)

In [138]:
#inventor, abstract dict 생성
dict_inventor_abstract = {}

for i in range(len(inventor)):
    dict_inventor_abstract[inventor[i]] = abstract[i]

In [139]:
len(dict_inventor_abstract)

379

In [140]:
len(drop_inventor)

280

In [142]:
for k,v in dict_inventor_abstract.items():
    if v=='':
        drop_inventor.append(k)

In [153]:
#inventor 그룹이 쓴 특허가 3개 이상인 것 dict 생성
inventor_abstract =dict_inventor_abstract.copy()

In [154]:
for k in set(drop_inventor):
    del inventor_abstract[k]

In [158]:
for k,v in inventor_abstractl.items():
    f = open("./"+k+".txt", 'w')
    f.write(v)
    f.close()